In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from numpy.random import multivariate_normal as mvnorm
from numpy.linalg import norm

import matplotlib.pyplot as plt
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
%matplotlib inline

import pdb

seed = 0
np.random.seed(seed)

### Import and generate data

In [2]:
# Import data
data = pd.read_pickle("~/Downloads/integrated_draws_20_5_4.pkl")
n_draws, n_time, dim_X, dim_fac = data['beta'].shape
beta = data['beta']
X_sd = data['x_sd']

# Generate X
m_fac = np.zeros((dim_fac))
s_fac = np.diag(np.ones((dim_fac)))
fac = np.expand_dims(mvnorm(m_fac, s_fac, (n_draws, n_time)), -1)

X = np.matmul(beta, fac).squeeze()

m_eps = np.zeros((dim_X))
for i in range(n_draws):
    eps = mvnorm(m_eps, np.diag(X_sd[i] ** 2), n_time)
    X[i] += X[i] + eps
    
X = X.transpose((2, 0, 1))
X_cov = np.einsum('ijkl,jmkl->imkl', np.expand_dims(X, 1), np.expand_dims(X, 0))

### Train

In [ ]:
from regain.covariance import ConstrainedTimeGraphicalLasso
tgl = ConstrainedTimeGraphicalLasso(max_iter=100, loss='DT', c_level=0.25, rho=5*1e1, psi="laplacian")
emp_inv_score, baseline_score, fit_score, pre = tgl.fit_cov(X_cov).eval_cov_pre()    

> /Users/sikai/Dropbox/Research/post_sum/code/regain/regain/covariance/graphical_lasso_.py(93)dtrace_constraint()
-> sqrt_term = np.sqrt(np.sum(pre_diag) ** 2 - 4 * (e - f + g) * -constraint)
(Pdb) print(np.sum(pre_diag) ** 2)
0.0003452474316207005
(Pdb) print(4 * (e - f + g) * -constraint)
0.000541478938765576
(Pdb) loss
0.01880974888248334
(Pdb) constraint
-0.5913567591292974


In [10]:
%debug

> /Users/sikai/Dropbox/Research/post_sum/code/regain/regain/covariance/constrained_time_graphical_lasso.py(82)loss_dtrace()
     80         loss_t, root_n, root_p = dtrace_constraint(S[i], K[i], C[i])
     81         loss.append(loss_t)
---> 82         if root_n != root_p:
     83             pdb.set_trace()
     84         # dt1.append(trace_squared_term)

ipdb> root_n
array([1.30160646, 1.01150467, 1.05058952, 2.99586376, 3.8153693 ,
       4.04896579, 5.08888365, 2.64631859, 4.06279915, 4.32146775,
       4.7549685 , 3.50401953, 5.70696221, 6.17332203, 3.14721179,
       3.38145471, 5.45706291, 5.57429919, 5.26467177, 4.40410817])
ipdb> q


In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
fig.patch.set_facecolor('white')
# colors = "bgrcmykw"
# color_index = 0
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.tab20.colors)

ax.plot(range(n_time), emp_inv_score, label=r'Empirical Inverse')
ax.plot(range(n_time), baseline_score, label=r'Constraint')
ax.plot(range(n_time), fit_score, label=r'TGL')

fig.legend(fontsize=15)
ax.set_ylabel('DTrace Loss', fontsize=15)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.set_xlabel('Time t', fontsize=15)
ax.set_title(r'DTrace Loss for Empirical Inverse, Constraint, TGL', fontsize=20)
plt.tight_layout()
# plt.savefig('diff_like_{}_{}.pdf'.format(dim_X, dim_fac))

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
fig.patch.set_facecolor('white')
# colors = "bgrcmykw"
# color_index = 0
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.tab20.colors)

ax.plot(range(n_time), emp_inv_score, label=r'Empirical Inverse')
ax.plot(range(n_time), baseline_score, label=r'Constraint')
ax.plot(range(n_time), fit_score, label=r'TGL')

fig.legend(fontsize=15)
ax.set_ylabel('DTrace Loss', fontsize=15)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.set_xlabel('Time t', fontsize=15)
ax.set_title(r'DTrace Loss for Empirical Inverse, Constraint, TGL', fontsize=20)
plt.tight_layout()
# plt.savefig('diff_like_{}_{}.pdf'.format(dim_X, dim_fac))

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
fig.patch.set_facecolor('white')
# colors = "bgrcmykw"
# color_index = 0

ax.plot(range(n_time), [sum(sum(abs(tgl.emp_inv[k]) > 0)) for k in range(n_time)], label=r'Empirical Inverse')
ax.plot(range(n_time), [sum(sum(abs(pre[k]) > 0)) for k in range(n_time)], label=r'TGL')

fig.legend(fontsize=15)
ax.set_ylabel('Sparsity', fontsize=15)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.set_xlabel('Time t', fontsize=15)
ax.set_title(r'Sparsity for Empirical Inverse and TGL', fontsize=20)
plt.tight_layout()
# plt.savefig('diff_supp_{}_{}.pdf'.format(dim_X, dim_fac))

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
fig.patch.set_facecolor('white')
# colors = "bgrcmykw"
# color_index = 0
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.tab20.colors)

diff_fit = [norm(pre[k] - pre[k-1], 'fro') for k in range(1, 100)]
diff_emp = [norm(tgl.emp_inv[k] - tgl.emp_inv[k-1], 'fro') for k in range(1, 100)]
ax.plot(range(1, n_time), np.array(diff_fit), alpha=1, label=r'Empirical Inverse')
ax.plot(range(1, n_time), np.array(diff_emp), alpha=1, label=r'TGL')

        
fig.legend(fontsize=15)
ax.set_ylabel('Difference in Frobenius Norm', fontsize=15)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.set_xlabel('Time t', fontsize=15)
ax.set_title(r'Difference in Frobenius Norm for Empirical Inverse and TGL', fontsize=20)
plt.tight_layout()
# plt.savefig('diff_fro_{}_{}.pdf'.format(dim_X, dim_fac))